In [1]:
import cv2
import numpy as np
import pyrealsense2 as rs

# Set up RealSense stream
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

try:
    while True:
        # Get frames
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        frame = np.asanyarray(color_frame.get_data())
        height, width, _ = frame.shape

        # Crop bottom half
        roi = frame[height//2:, :]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)

        # Invert binary threshold (black lines on white background)
        _, binary = cv2.threshold(blur, 60, 255, cv2.THRESH_BINARY_INV)

        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        centers = []

        for c in contours:
            M = cv2.moments(c)
            if M["m00"] > 500:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                centers.append(cx)
                cv2.circle(roi, (cx, cy), 5, (0, 255, 0), -1)

        # Draw lane center
        if len(centers) >= 2:
            lane_center = int(np.mean(centers))
            cv2.line(roi, (lane_center, 0), (lane_center, roi.shape[0]), (255, 0, 0), 2)
            cv2.putText(roi, f"Lane Center: {lane_center}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

        cv2.imshow("Lane Detection", roi)
        if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/kartik/.local/lib/python3.10/site-packages/cv2/qt/plugins"


KeyboardInterrupt: 